N = 4 <br>
prototype

<h4>Item <i>"a)"</i> Onde f = 0 e u(x,y)= alpha * np.exp(x) * np.sin(y) na fronteira</h4>

In [39]:
import numpy as np
from time import time

In [58]:
N=2**7
MAXITER=10**3

alpha = 3.0
h=1./N

TOL = 10**-5 * h

<h3>Funções Auxiliares</h3>

<h4>Jacobi</h4>

In [59]:
def jacobi_method(U_old, U_new, h, F):
    for i in range(1,N):
        for j in range(1,N):
            U_new[i][j] = 0.25*(U_old[i-1][j] + U_old[i+1][j] + U_old[i][j-1] + U_old[i][j+1] + h**2 * F[i][j])


<h4>PASSO 1 e 2</h4>

In [60]:
U_old = np.zeros((N+1,N+1))
U_new = np.zeros((N+1,N+1))
F = np.zeros((N+1,N+1)) 

<h4>PASSO 3</h4>

In [61]:
F = np.zeros((N+1,N+1)) 

<h4>PASSO 4</h4>

In [62]:
# Atualiza as bordas


# Parte inferior e superior da Malha
for i in range(N+1):
    U_old[i, 0] = alpha * np.exp(h*i) * np.sin(h*0.)
    U_old[i, N] = alpha * np.exp(h*i) * np.sin(h*N)

# Parte esquerda e direita da Malha
for j in range(N+1):
    U_old[0, j] = alpha * np.exp(h*0.) * np.sin(h*j)
    U_old[N, j] = alpha * np.exp(h*N) * np.sin(h*j)    


In [63]:
U_old

array([[0.        , 0.02343726, 0.04687309, ..., 2.49877916, 2.51167271,
        2.52441295],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.54421217],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.56416667],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        6.75567939],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        6.80866484],
       [0.        , 0.06370908, 0.12741428, ..., 6.792386  , 6.82743428,
        6.86206586]])

<h4>PASSO 5-7</h4>

In [64]:
for it in range(MAXITER):
    jacobi_method(U_old, U_new, h, F)
    
#     if calculate_diff(U_new, U_old) <= TOL:
#         print('Algoritmo convergiu com %d iterações' %(it))
#         break
    
    U_old = U_new.copy()

In [65]:
U_new

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.11366552e-07, 8.22708553e-07, ...,
        6.67469278e-06, 3.34180368e-06, 0.00000000e+00],
       [0.00000000e+00, 8.20270273e-07, 1.64049147e-06, ...,
        1.33145548e-05, 6.67895309e-06, 0.00000000e+00],
       ...,
       [0.00000000e+00, 2.22968152e-06, 4.45922859e-06, ...,
        2.84547575e-05, 1.42746806e-05, 0.00000000e+00],
       [0.00000000e+00, 1.11818844e-06, 2.23630963e-06, ...,
        1.42748387e-05, 7.14372650e-06, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])